# 1

# 2

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

In [92]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

In [93]:
AUC=pd.read_csv("AUCs.txt",'\t')
AUC=AUC.rename(columns={'Unnamed: 0': "names"})

AUC

,names,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [94]:

corr, p =wilcoxon(AUC["C4.5"],AUC["C4.5+cf"],correction=True)
corr ,p

(43.0, 0.888806954437247)

In [95]:
print corr
print p

43.0
0.888806954437


In [96]:
AUC.columns

Index([u'names', u'C4.5', u'C4.5+m', u'C4.5+cf', u'C4.5+m+cf'], dtype='object')

In [97]:
columns=AUC.columns[1:]

In [98]:
corr_data=[]
for i,fst in enumerate(columns):
    for j,second in enumerate(columns):
        if i>=j:
            continue
        corr, p =wilcoxon(AUC[fst],AUC[second],correction=True)
        corr_data.append([fst,second,corr,p])

In [99]:
corr_data

[['C4.5', 'C4.5+m', 6.5, 0.012030358470209076],
 ['C4.5', 'C4.5+cf', 43.0, 0.888806954437247],
 ['C4.5', 'C4.5+m+cf', 11.0, 0.017496137183767736],
 ['C4.5+m', 'C4.5+cf', 17.0, 0.050300934429293433],
 ['C4.5+m', 'C4.5+m+cf', 22.0, 0.35029075975557733],
 ['C4.5+cf', 'C4.5+m+cf', 10.0, 0.025369859822053694]]

In [100]:
param_correlation = pd.DataFrame.from_records(corr_data)
param_correlation.columns = ['product_A', 'product_B', 'correlation', 'p']

In [101]:
param_correlation.correlation.min()

6.5

In [102]:
param_correlation

,product_A,product_B,correlation,p
0,C4.5,C4.5+m,6.5,0.012030
1,C4.5,C4.5+cf,43.0,0.888807
2,C4.5,C4.5+m+cf,11.0,0.017496
3,C4.5+m,C4.5+cf,17.0,0.050301
4,C4.5+m,C4.5+m+cf,22.0,0.350291
5,C4.5+cf,C4.5+m+cf,10.0,0.025370


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

Ответ: C4.5; C4.5+m

In [103]:
(param_correlation.p<0.05).value_counts()

True     3
False    3
Name: p, dtype: int64

Ответ:3

# 4

судя по данным из преведущего опроса настройка какого параметра даёт наибольшее увеличение качества?

# 5

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [104]:
reject, p_corrected, a1, a2=multipletests(param_correlation.p, alpha=0.05,method='holm')

In [105]:
param_correlation['p_corrected']=p_corrected
param_correlation['reject']=reject

In [106]:
param_correlation

,product_A,product_B,correlation,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.012030,0.072182,False
1,C4.5,C4.5+cf,43.0,0.888807,0.888807,False
2,C4.5,C4.5+m+cf,11.0,0.017496,0.087481,False
3,C4.5+m,C4.5+cf,17.0,0.050301,0.150903,False
4,C4.5+m,C4.5+m+cf,22.0,0.350291,0.700582,False
5,C4.5+cf,C4.5+m+cf,10.0,0.025370,0.101479,False


In [107]:
(param_correlation.p_corrected<0.05).value_counts()

False    6
Name: p_corrected, dtype: int64

# 6

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [108]:
reject, p_corrected, a1,a2=multipletests(param_correlation.p, alpha=0.05,method='fdr_bh')

array([False, False, False, False, False, False], dtype=bool)

In [82]:
param_correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [84]:
param_correlation[param_correlation.reject==True].sort_values(by='correlation',ascending=False).head()

,product_A,product_B,correlation,p,p_corrected,reject
